In [13]:
# %load train_kronnc.py
# random del 15 个nodes  email.txt
import torch
import pickle
import random
import numpy as np
import networkx as nx 
from networkx.convert import from_dict_of_dicts
from networkx.classes.graph import Graph
from kronEM import *
seed =1900
np.random.seed(seed)
random.seed(seed)

adj_address ="/data/chenmy/voter/seed1051email128-adjmat.pickle"
with open(adj_address,'rb') as f:
    objective_adj = pickle.load(f,encoding='latin1')
objective_adj = np.array(objective_adj)

In [14]:
Ground_truth_adj = torch.FloatTensor(objective_adj)
sz = Ground_truth_adj.shape[0]
k = int(np.log2(sz))
remove_proportion = 0.01
del_num = int(sz*remove_proportion)
mask_un_obs,mask_obs,miss_idx = missing_label(sz,remove_proportion)
G = Ground_truth_adj*mask_obs
nG = G.data.numpy()
H = G.clone()
# print(2,(abs(H.data.numpy()-objective_adj)*mask_obs.data.numpy()).sum())
init_z = Ground_truth_adj*mask_un_obs
missing_edges = int(init_z.sum())
print("miss nodes ",miss_idx)
# initial partial z with fixed missing edges num
z_ele_num = int(mask_un_obs.sum())
z_element_choice = torch.randperm(z_ele_num)[:missing_edges]
z_element = torch.nonzero(mask_un_obs)
init_z_edges = torch.index_select(z_element,0,z_element_choice)
H[init_z_edges[:,0],init_z_edges[:,1]] = 1
# print(abs((H - Ground_truth_adj)*mask_obs).sum())
# print(G,H,z_element_choice)
# initial kronecker
p0 = torch.FloatTensor([[0.9,0.7],[0.7, 0.3]])#torch.FloatTensor([[0.4408, 0.1770],[0.4951, 0.2585]])  # 
generator = kronecker_Generator(p0,k,2)
Pk = generator.generator_adjacency()
init_H = H.data.numpy()
label_non_obs = mask_un_obs.data.numpy()
init_Pk = Pk.detach().data.numpy()
perm = np.arange(sz)
print(abs((init_H- objective_adj)*mask_obs.data.numpy()).sum())
init_label_non_obs= label_non_obs.copy()

miss nodes  tensor([59])
0.0


In [18]:
# %load kronEM.py
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,auc
import numpy as np

def kronecker(A,B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))
class kronecker_Generator(nn.Module):
    def __init__(self,p0,korder = 3,node_num = 2):
        super(kronecker_Generator,self).__init__()
        self.p = Parameter(p0,requires_grad = True)
        # self.p = Parameter(torch.rand(node_num,node_num,requires_grad=True))
        self.korder = korder
        # print(self.p)
    def generator_adjacency(self):
        k = self.korder
        p0 = self.p
        adj = self.p
        for i in range(k-1):
            adj = kronecker(adj,p0)
        return adj
def loss_func(sigma,Pk):
    loss = -torch.sum((1-sigma)*torch.log(1-Pk)+sigma*torch.log(Pk))
    return loss
def metropolis_update_ratio(sigma_before,sigma_later,Pk):
    '''
    if memory is sufficinet, this one is much cleaner to execute
    '''
    Nll_before=(1-sigma_before)*np.log(1-Pk) +sigma_before*np.log(Pk)
    Nll_later=(1-sigma_later)*np.log(1-Pk) +sigma_later*np.log(Pk)
    ratio=np.exp(np.sum(Nll_later-Nll_before))
    return ratio
def SwapElement(sigma_before,i,j):
    i_topology=sigma_before[i,:]
    j_topology=sigma_before[j,:]
    sigma_later=np.copy(sigma_before)
    sigma_later[i,:]=j_topology
    sigma_later[j,:]=i_topology
    sigma_later[:,i]=sigma_before[:,j]
    sigma_later[:,j]=sigma_before[:,i]
    sigma_later[i,j]=sigma_before[i,j]
    sigma_later[j,i]=sigma_before[j,i]
    sigma_later[i,i]=sigma_before[j,j]
    sigma_later[j,j]=sigma_before[i,i]
    return sigma_later

def SamplePermutation(Pk,sigma,u,n1_swap,n2_swap,label_non_obs,obj_adj):
    sigma_later=SwapElement(sigma,n1_swap, n2_swap)
    ratio=metropolis_update_ratio(sigma,sigma_later,Pk)
    if u<ratio:
        sigma=sigma_later
        label_non_obs=SwapElement(label_non_obs,n1_swap, n2_swap)
        obj_adj = SwapElement(obj_adj,n1_swap,n2_swap)
#         print(ifswap)
    return sigma,label_non_obs,obj_adj
def SampleZ(H,Pk,label_non_obs,u):
    mat_size=len(Pk)
    edge_in_non_obs=(H>0)*label_non_obs  
    edge_position=np.where(edge_in_non_obs)
    edge_removed=np.random.randint(len(edge_position[0]))
    px=Pk[edge_position[0][edge_removed],edge_position[1][edge_removed]]
    non_edge_in_non_obs=(H<1)*label_non_obs
    ##return 
    py_array=non_edge_in_non_obs*Pk
    py_array=np.ravel(py_array)
    py=np.random.choice(range(len(py_array)), size=1, p=py_array/np.sum(py_array))
    ratio=(1-py_array[py])/(1-px)
    if ratio<u:
        H[py//mat_size,py%mat_size]=1
        H[edge_position[0][edge_removed],edge_position[1][edge_removed]]=0
        # print('accept')
    return H

def missing_label(sz,missing_percent):
     # random sample
    missing_num = int(sz*missing_percent)
    idx = torch.randperm(sz)[:missing_num]
    mask_un_obs = torch.zeros(sz,sz)
    for i in idx:
        mask_un_obs[i] = 1
        mask_un_obs[:,i:i+1] = 1
    mask_obs = 1- mask_un_obs
    return mask_un_obs,mask_obs,idx
# only sigma and kronecker pk are itered
def E_step(sigma,N,warmup,Pk,label_non_obs,obj_adj):
    u1=np.random.rand(N+warmup)
    u2=np.random.rand(N+warmup)
    sigma_hist=[]
    Z_label=[]
    Node_list=np.arange(len(Pk))
    element_to_swap=np.random.choice(a=Node_list,size=(2,3*(N+warmup)))
    mask=element_to_swap[1,:]!=element_to_swap[0,:]# it is pointless to swap the same element
    n1_swap=element_to_swap[0,:][mask]
    n2_swap=element_to_swap[1,:][mask]
   
    for i in range(N+warmup):
        # if i%100==0:
        #     print("E_step",i,round(i/(N+warmup),4))
        sigma=SampleZ(sigma,Pk,label_non_obs,u1[i]) # sigma shuffle后的adj
        # print("n1,n2",n1_swap,n2_swap,mask)
        sigma,label_non_obs,obj_adj=SamplePermutation(Pk,sigma,u2[i],n1_swap[i],n2_swap[i],label_non_obs,obj_adj)
#         if ifswap:
#             perm[n1_swap[i]],perm[n2_swap[i]] = perm[n2_swap[i]],perm[n1_swap[i]]
#             print(ifswap,n1_swap[i],n2_swap[i],perm)
        if i>=warmup:
            sigma_hist.append(sigma)
            Z_label.append(label_non_obs)
    return sigma_hist,Z_label,obj_adj #（H+G）

def M_step(epoch,sigma_train,p0,k,N):
    losses = []
    generator = kronecker_Generator(p0,k,2)
    learning_rate = 1e-5#0.0000001
    opt_net = optim.SGD(generator.parameters(),lr = learning_rate)
    decayRate = 0.95

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=opt_net, gamma=decayRate) 
    for i in range(epoch):   
        opt_net.zero_grad()
        Pk = generator.generator_adjacency()
        loss = loss_func(sigma_train,Pk,N)
        loss2 = loss2_func(sigma_train.detach().numpy(),Pk.detach().numpy(),N)
        loss.backward()
        losses.append(loss.item())
        # print(str(i),loss.item(),loss2.item(),(loss2-loss).item())
        opt_net.step()
        scheduler.step()
        for group in opt_net.param_groups:
            for param in group["params"]: 
              # print("before param",param)
              param.data.clamp_(0.0001,0.9999)
              # print("after param",param)
    
        for p in generator.parameters():
            p0 = p.data  
        generator = kronecker_Generator(p0,k,2)
        Pk = generator.generator_adjacency()
        Pk = Pk.detach().numpy() 
        # evaluation
        #   nll infer  nll true
    return np.mean(losses),Pk,p0

def kronEM(iterstep,N,warmup,H,Pk,label_non_obs,epoch,p0,obj_adj,k):
    emlosses = []
    for i in range(iterstep):
        print("start iterstep",i)
        sigma_hist,Z_label,obj_adj = E_step(H,N,warmup,Pk,label_non_obs,obj_adj)
        print("E-step")
        H = sigma_hist[-1]
        label_non_obs = Z_label[-1]
        sigma_hist_train = np.array(sigma_hist)
        sigma_train = torch.DoubleTensor(sigma_hist_train)
        emloss,Pk,p0 = M_step(epoch,sigma_train,p0,k,N)
        emlosses.append(emloss)
        print("*************\n EM loss is %f"%emloss,"p0 is ",p0)
        
        # evaluation
        inferp_nll = NLL(H,Pk)
        print("inferp_nll is %f "%(inferp_nll))
        obs_mask = (1-label_non_obs).astype(bool)
        obs_auc = calauc(H,Pk,obs_mask)
        non_obs_auc = calauc(H,Pk,label_non_obs.astype(bool))
        #         print("perm is ", perm)
        
        label_obs = (1-label_non_obs)
        obs_diff_edge = abs((H - obj_adj)*label_obs).sum()
        unobs_diff_edge =  abs((H - obj_adj)*label_non_obs).sum()
        all_diff_edge = abs((H - obj_adj)).sum()
        # D_abs = torch.abs(orip-p0).mean()
        print("infer p is ",p0,  "obseved auc  is %f and non_obs auc is %f and obs_diff_edge is %f and un_obs_diff_edge %f and all_diff_edge is %f"%(obs_auc,non_obs_auc,obs_diff_edge,unobs_diff_edge,all_diff_edge))
    
    return emlosses,perm,H

def NLL(sigma_true,pk):
  Nll_before=(1-sigma_true)*np.log(1-pk) +sigma_true*np.log(pk)
  return -np.sum(Nll_before)

def loss2_func(sigma_train,Pk,N):
  loss2 = 0
  for i in range(N):
    loss2+= NLL(sigma_train[i],Pk)
    # print(loss2)
  return loss2/N

def loss_func(sigma,Pk,N):
    loss = -torch.sum((1-sigma)*torch.log(1-Pk)+sigma*torch.log(Pk))
    return loss/N

def generator_adj(korder,p):
    k = korder
    p0 = p
    adj = p
    for i in range(k-1):
        adj = kronecker(adj,p0)
    return adj
def calauc(H,Pk,mask):
    fpr, tpr, thresholds = roc_curve(H[mask],Pk[mask])
    Auc = auc(fpr, tpr)
    return Auc

In [19]:
warmup=100
N = 1 
iterstep = 100
epoch = 10
emlosses2,perm2,H = kronEM(iterstep,N,warmup,init_H,init_Pk,label_non_obs,epoch,p0,objective_adj,k)

start iterstep 0
E-step
*************
 EM loss is 4063.747371 p0 is  tensor([[0.9169, 0.7380],
        [0.7381, 0.3137]])
inferp_nll is 4057.917969 
infer p is  tensor([[0.9169, 0.7380],
        [0.7381, 0.3137]]) obseved auc  is 0.591916 and non_obs auc is 0.979249 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 1
E-step
*************
 EM loss is 3852.957363 p0 is  tensor([[0.9123, 0.7338],
        [0.7341, 0.3284]])
inferp_nll is 3850.409180 
infer p is  tensor([[0.9123, 0.7338],
        [0.7341, 0.3284]]) obseved auc  is 0.642987 and non_obs auc is 0.989130 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 2
E-step
*************
 EM loss is 3695.876756 p0 is  tensor([[0.9095, 0.7306],
        [0.7309, 0.3367]])
inferp_nll is 3694.953613 
infer p is  tensor([[0.9095, 0.7306],
        [0.7309, 0.3367]]) obseved auc  is 0.682308 and non_obs auc is 0.996047 and obs_diff_edge i

E-step
*************
 EM loss is 3414.521417 p0 is  tensor([[0.9217, 0.7150],
        [0.7159, 0.3434]])
inferp_nll is 3414.507568 
infer p is  tensor([[0.9217, 0.7150],
        [0.7159, 0.3434]]) obseved auc  is 0.762697 and non_obs auc is 0.980237 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 24
E-step
*************
 EM loss is 3422.087987 p0 is  tensor([[0.9226, 0.7148],
        [0.7157, 0.3431]])
inferp_nll is 3422.077148 
infer p is  tensor([[0.9226, 0.7148],
        [0.7157, 0.3431]]) obseved auc  is 0.761464 and non_obs auc is 0.977273 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 25
E-step
*************
 EM loss is 3420.266660 p0 is  tensor([[0.9235, 0.7147],
        [0.7155, 0.3427]])
inferp_nll is 3420.256836 
infer p is  tensor([[0.9235, 0.7147],
        [0.7155, 0.3427]]) obseved auc  is 0.761284 and non_obs auc is 0.980237 and obs_diff_edge is 0.000000 and 

E-step
*************
 EM loss is 3380.419877 p0 is  tensor([[0.9303, 0.7211],
        [0.7214, 0.3239]])
inferp_nll is 3380.412109 
infer p is  tensor([[0.9303, 0.7211],
        [0.7214, 0.3239]]) obseved auc  is 0.769307 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 47
E-step
*************
 EM loss is 3371.977689 p0 is  tensor([[0.9304, 0.7215],
        [0.7219, 0.3228]])
inferp_nll is 3371.963867 
infer p is  tensor([[0.9304, 0.7215],
        [0.7219, 0.3228]]) obseved auc  is 0.772757 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 48
E-step
*************
 EM loss is 3374.693701 p0 is  tensor([[0.9306, 0.7219],
        [0.7222, 0.3220]])
inferp_nll is 3374.684814 
infer p is  tensor([[0.9306, 0.7219],
        [0.7222, 0.3220]]) obseved auc  is 0.771444 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and 

E-step
*************
 EM loss is 3322.719141 p0 is  tensor([[0.9343, 0.7296],
        [0.7296, 0.3032]])
inferp_nll is 3322.693848 
infer p is  tensor([[0.9343, 0.7296],
        [0.7296, 0.3032]]) obseved auc  is 0.781499 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 70
E-step
*************
 EM loss is 3318.191742 p0 is  tensor([[0.9339, 0.7304],
        [0.7304, 0.3018]])
inferp_nll is 3318.158203 
infer p is  tensor([[0.9339, 0.7304],
        [0.7304, 0.3018]]) obseved auc  is 0.783180 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 71
E-step
*************
 EM loss is 3324.543463 p0 is  tensor([[0.9337, 0.7310],
        [0.7311, 0.3008]])
inferp_nll is 3324.525391 
infer p is  tensor([[0.9337, 0.7310],
        [0.7311, 0.3008]]) obseved auc  is 0.781654 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and 

E-step
*************
 EM loss is 3316.354688 p0 is  tensor([[0.9320, 0.7353],
        [0.7353, 0.2951]])
inferp_nll is 3316.347656 
infer p is  tensor([[0.9320, 0.7353],
        [0.7353, 0.2951]]) obseved auc  is 0.784903 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 93
E-step
*************
 EM loss is 3313.396400 p0 is  tensor([[0.9319, 0.7356],
        [0.7356, 0.2944]])
inferp_nll is 3313.389648 
infer p is  tensor([[0.9319, 0.7356],
        [0.7356, 0.2944]]) obseved auc  is 0.785410 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and un_obs_diff_edge 4.000000 and all_diff_edge is 4.000000
start iterstep 94
E-step
*************
 EM loss is 3313.902489 p0 is  tensor([[0.9319, 0.7359],
        [0.7359, 0.2939]])
inferp_nll is 3313.898438 
infer p is  tensor([[0.9319, 0.7359],
        [0.7359, 0.2939]]) obseved auc  is 0.785403 and non_obs auc is 1.000000 and obs_diff_edge is 0.000000 and 

In [20]:
kronfit 已知部分 + gumbel   + 两个loss （ll+states loss）

0.0

In [13]:
perm

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  85,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  65,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        37,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  22,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127])

In [16]:
permutation_mat  = np.eye(H.shape[0])[perm]
shuffle_H = np.dot(np.dot(permutation_mat,objective_adj),permutation_mat.T)

In [10]:
sigma_later1=SwapElement(init_H,37,65)

In [8]:
sigma_later=SwapElement(sigma_later,22,85)

In [12]:
(abs(sigma_later1-H)*mask_obs.data.numpy()).sum()

68.0

In [18]:
(abs(shuffle_H-H)*mask_obs.data.numpy()).sum()

0.0